# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 10:14AM,241184,10,0085837391,ISDIN Corporation,Released
1,Jul 6 2022 10:14AM,241184,10,0085837480,ISDIN Corporation,Released
2,Jul 6 2022 10:14AM,241184,10,0085837481,ISDIN Corporation,Released
3,Jul 6 2022 10:14AM,241184,10,0085837421,ISDIN Corporation,Released
4,Jul 6 2022 10:14AM,241184,10,0085837578,ISDIN Corporation,Released
5,Jul 6 2022 10:14AM,241184,10,0085837661,ISDIN Corporation,Released
6,Jul 6 2022 10:14AM,241184,10,0085837670,ISDIN Corporation,Released
7,Jul 6 2022 10:14AM,241184,10,0085837732,ISDIN Corporation,Released
8,Jul 6 2022 10:14AM,241184,10,0085837733,ISDIN Corporation,Released
9,Jul 6 2022 10:14AM,241184,10,0085837683,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
66,241178,Released,2
67,241180,Released,1
68,241182,Released,46
69,241183,Released,1
70,241184,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241178,NaN,1.0,2.0
241180,NaN,NaN,1.0
241182,NaN,NaN,46.0
241183,NaN,NaN,1.0
241184,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241062,14.0,14.0,1.0
241069,0.0,20.0,35.0
241070,0.0,0.0,49.0
241074,0.0,0.0,41.0
241079,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241062,14,14,1
241069,0,20,35
241070,0,0,49
241074,0,0,41
241079,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241062,14,14,1
1,241069,0,20,35
2,241070,0,0,49
3,241074,0,0,41
4,241079,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241062,14,14,1
1,241069,,20,35
2,241070,,,49
3,241074,,,41
4,241079,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation
14,Jul 6 2022 10:14AM,241182,10,ISDIN Corporation
60,Jul 6 2022 10:10AM,241183,21,"NBTY Global, Inc."
61,Jul 6 2022 9:46AM,241180,10,Eywa Pharma Inc.
62,Jul 6 2022 9:31AM,241178,12,Hush Hush
65,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC"
68,Jul 6 2022 9:24AM,241175,12,Hush Hush
78,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated"
79,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.
111,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,,,14
1,Jul 6 2022 10:14AM,241182,10,ISDIN Corporation,,,46
2,Jul 6 2022 10:10AM,241183,21,"NBTY Global, Inc.",,,1
3,Jul 6 2022 9:46AM,241180,10,Eywa Pharma Inc.,,,1
4,Jul 6 2022 9:31AM,241178,12,Hush Hush,,1,2
5,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",,,3
6,Jul 6 2022 9:24AM,241175,12,Hush Hush,,,10
7,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",,,1
8,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,,,32
9,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,14,,
1,Jul 6 2022 10:14AM,241182,10,ISDIN Corporation,46,,
2,Jul 6 2022 10:10AM,241183,21,"NBTY Global, Inc.",1,,
3,Jul 6 2022 9:46AM,241180,10,Eywa Pharma Inc.,1,,
4,Jul 6 2022 9:31AM,241178,12,Hush Hush,2,1,
5,Jul 6 2022 9:25AM,241177,10,"Nextsource Biotechnology, LLC",3,,
6,Jul 6 2022 9:24AM,241175,12,Hush Hush,10,,
7,Jul 6 2022 9:18AM,241174,10,"Senseonics, Incorporated",1,,
8,Jul 6 2022 9:18AM,241169,20,Alumier Labs Inc.,32,,
9,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,14,,
1,Jul 6 2022 10:14AM,241182,10,ISDIN Corporation,46,,
2,Jul 6 2022 10:10AM,241183,21,"NBTY Global, Inc.",1,,
3,Jul 6 2022 9:46AM,241180,10,Eywa Pharma Inc.,1,,
4,Jul 6 2022 9:31AM,241178,12,Hush Hush,2,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,14.0,NaN,NaN
1,Jul 6 2022 10:14AM,241182,10,ISDIN Corporation,46.0,NaN,NaN
2,Jul 6 2022 10:10AM,241183,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 6 2022 9:46AM,241180,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Jul 6 2022 9:31AM,241178,12,Hush Hush,2.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 10:14AM,241184,10,ISDIN Corporation,14.0,0.0,0.0
1,Jul 6 2022 10:14AM,241182,10,ISDIN Corporation,46.0,0.0,0.0
2,Jul 6 2022 10:10AM,241183,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 6 2022 9:46AM,241180,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Jul 6 2022 9:31AM,241178,12,Hush Hush,2.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,8439736,605.0,22.0,3.0
12,1205666,29.0,1.0,0.0
16,1446725,11.0,4.0,0.0
18,482320,2.0,0.0,0.0
19,723389,59.0,43.0,2.0
20,1446622,119.0,25.0,28.0
21,241183,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,8439736,605.0,22.0,3.0
1,12,1205666,29.0,1.0,0.0
2,16,1446725,11.0,4.0,0.0
3,18,482320,2.0,0.0,0.0
4,19,723389,59.0,43.0,2.0
5,20,1446622,119.0,25.0,28.0
6,21,241183,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,605.0,22.0,3.0
1,12,29.0,1.0,0.0
2,16,11.0,4.0,0.0
3,18,2.0,0.0,0.0
4,19,59.0,43.0,2.0
5,20,119.0,25.0,28.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,605.0
1,12,Released,29.0
2,16,Released,11.0
3,18,Released,2.0
4,19,Released,59.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,2.0,28.0,0.0
Executing,22.0,1.0,4.0,0.0,43.0,25.0,0.0
Released,605.0,29.0,11.0,2.0,59.0,119.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,2.0,28.0,0.0
1,Executing,22.0,1.0,4.0,0.0,43.0,25.0,0.0
2,Released,605.0,29.0,11.0,2.0,59.0,119.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,3.0,0.0,0.0,0.0,2.0,28.0,0.0
1,Executing,22.0,1.0,4.0,0.0,43.0,25.0,0.0
2,Released,605.0,29.0,11.0,2.0,59.0,119.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()